In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import joblib
import tensorflow as tf

# Definindo os parâmetros para os dados de teste
num_test_samples = 200  # Número de amostras de teste
velocidade_test = np.random.randint(0, 101, num_test_samples)
velocidade_desejada_test = np.random.randint(0, 101, num_test_samples)
felicidade_equipe_test = np.random.choice([10, 7.5, 5, 2.5], num_test_samples)
felicidade_cliente_test = np.random.choice([10, 7.5, 5, 2.5], num_test_samples)
confianca_entrega_test = np.random.choice([10, 7.5, 5, 2.5], num_test_samples)
total_pessoas_equipe_test = np.random.randint(0, 11, num_test_samples)
horas_extras_test = np.random.choice(['sim', 'não'], num_test_samples)

# Codificação one-hot para horas extras
horas_extras_encoded_test = pd.get_dummies(horas_extras_test)

# Criando o DataFrame para dados de teste
test_data = pd.DataFrame({
    'velocidade': velocidade_test,
    'velocidade_desejada': velocidade_desejada_test,
    'felicidade_equipe': felicidade_equipe_test,
    'felicidade_cliente': felicidade_cliente_test,
    'confianca_entrega': confianca_entrega_test,
    'total_pessoas_equipe': total_pessoas_equipe_test,
    'horas_extras_sim': horas_extras_encoded_test['sim'],
    'horas_extras_nao': horas_extras_encoded_test['não']
})

# Exibindo as primeiras linhas do DataFrame de teste
print(test_data.head())

# Convertendo o DataFrame em um array numpy
test_data_array = test_data.to_numpy()
print(test_data_array[:5])  # Mostrando as primeiras 5 linhas do array para verificação

# Carregar o escalador salvo
scaler = joblib.load('scaler.joblib')

# Escalar os dados de teste
test_data_scaled = scaler.transform(test_data_array)

# Transformar os dados para a forma [samples, time steps, features]
time_step = 3
X_test = []

for i in range(len(test_data_scaled) - time_step + 1):
    X_test.append(test_data_scaled[i:(i + time_step), :])

X_test = np.array(X_test)
print(f'X_test shape: {X_test.shape}')  # Esperado: (n_samples, time_steps, n_features)

# Carregar o modelo treinado no novo formato .keras
model = load_model('modelo_treinado.keras')

# Compilar o modelo novamente após carregar
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

# Definindo a função de previsão fora do loop
@tf.function(input_signature=[tf.TensorSpec(shape=[None, time_step, X_test.shape[2]], dtype=tf.float32)])
def predict_function(input_data):
    return model(input_data, training=False)

# Fazer previsões nos dados de teste
test_predictions = predict_function(tf.convert_to_tensor(X_test, dtype=tf.float32))

# Inverter a escala das previsões
test_predictions_inversed = scaler.inverse_transform(test_predictions.numpy())

# Exibir previsões
print("Previsões nos dados de teste:")
print(test_predictions_inversed)


2024-06-13 05:31:30.319514: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 05:31:30.328880: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 05:31:30.419423: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 05:31:32.883265: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


   velocidade  velocidade_desejada  felicidade_equipe  felicidade_cliente  \
0           7                   19                2.5                 2.5   
1          18                   27                2.5                 7.5   
2          73                    9                5.0                10.0   
3          99                   17                7.5                10.0   
4           9                   88               10.0                 2.5   

   confianca_entrega  total_pessoas_equipe  horas_extras_sim  horas_extras_nao  
0                7.5                    10              True             False  
1               10.0                     8              True             False  
2                5.0                     3              True             False  
3                2.5                     3             False              True  
4                7.5                     0              True             False  
[[7 19 2.5 2.5 7.5 10 True False]
 [18 27 2.5 7.5 1

/home/abu/Desenvolvimento/Gabriel/env/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


Previsões nos dados de teste:
[[48.71842    49.114334    6.4555206  ...  5.027375    0.60628027
   0.4164394 ]
 [45.54611    55.014072    6.0238028  ...  4.9120417   0.48555434
   0.5371782 ]
 [45.392715   49.271595    5.8260555  ...  5.5216923   0.468619
   0.520617  ]
 ...
 [52.31076    53.35618     6.8365097  ...  4.501988    0.6549139
   0.34863633]
 [51.055836   57.255024    6.155848   ...  4.969801    0.52811635
   0.4746403 ]
 [52.001278   57.93815     5.8121223  ...  5.2838063   0.52161676
   0.5059376 ]]
